In [1]:
%cd ..

/home/yongming/workspace/research/apperception


In [2]:
# Deleting the .apperception_cache if it exists, as to avoid DB conflict errors
import os
import shutil
import sys

sys.path.append(os.path.join(os.getcwd(),"apperception"))

dirpath = os.path.join('.apperception_cache')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

dirpath = os.path.join('output')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)
os.mkdir(dirpath)

# This piece of code is unsafe, and should not be run if not needed. 
# It serves for test purposes when one recieves a "dead kernel" error.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [13]:
##### NUSCENES DATA #####
from new_world import empty_world, World
from scenic_util import transformation
from camera import Camera
import pandas as pd
import pickle
print(os. getcwd())
World.db.reset()

name = 'ScenicWorld' # world name
units = 'metrics'      # world units
user_data_dir = os.path.join("v1.0-mini")


with open('./benchmarks/df_sample_data.pickle', "rb") as f:
    df_sample_data = pickle.loads(f.read())
with open('./benchmarks/df_annotation.pickle', "rb") as f:
    df_annotation = pickle.loads(f.read())


world = empty_world(name=name)

from camera_config import fetch_camera_config
# scenes = ["scene-0061", "scene-0103","scene-0553", "scene-0655", "scene-0757", "scene-0796", "scene-0916", "scene-1077", "scene-1094", "scene-1100"]
scenes = ["scene-0061"]
for scene in scenes:
    config = fetch_camera_config(scene, df_sample_data)
    camera = Camera(config=config, id=scene)
    world = world.add_camera(camera)
    df_config = df_sample_data[df_sample_data['scene_name'] == scene][['sample_token']]
    df_ann = df_annotation.join(df_config.set_index('sample_token'), on='sample_token', how='inner')
    world = world.recognize(camera, df_ann)


/home/yongming/workspace/research/apperception


In [14]:
cams = world.get_camera()
# ids = world.get_id()
# print("cameras are", cams[:2])
# print("ids are", ids)

execute: insert_cam
New camera inserted successfully.........
execute: retrieve_cam
execute: insert_bbox_traj
Recognization done, saving to database......
execute: get_cam
done execute node


In [ ]:
x = df_sample_data["heading"].unique()
x.sort()
x[x.shape[0] // 2 + 330:]

In [ ]:
world.road_direction(400, 1100)

In [ ]:
import time
start_time = time.time()
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40))

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")


filtered_world = filtered_world.filter_relative_to_type(lambda obj: (cam.x - 10) <= obj.x <= (cam.x + 10) 
                                                                          and (cam.y - 10) <= obj.y <= (cam.y + 0)
                                                                          and (cam.heading + 5) <= filtered_world.roadDirection(cam.x, cam.y) <= (cam.heading - 5))
### when using predicate
predicate(lambda obj, cam: any(obj.convert_to_cam) >= (-10, -10) 
              and any(obj.convert_to_cam) <= (10, 10)).get_camtimestamp()
'''
To realize this:
 1. def convert_to_camera(): return 'ConvertCamera(obj_table.traj, obj_table.cameraId)'
 2. any
 3. <= , >= will be convert to where clause
'''

Select * from item_general_trajectory as traj, cameras join on cameraId where any(convertCamera(traj.trajCentroid, cameras.ego_config)) >= (-10,-10)
                                                and any(convertCamera(traj.trajCentroid, traj.cameraId)) <= (10,10))

# filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
#                                                                             and 20 <= (obj.y - cam.y) <= 40
#                                                                             and obj.heading == cam.heading)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

print("----------------------------------------------------------------------")
print("Total execution time is: %s seconds" % (time.time() - start_time))
# print("Device Details: \n Processor: AMD Ryzen 7 5800H \n RAM Size: 16GB \n Graphics Card: NVIDIA GeForce RTX 3060 Laptop")
print("----------------------------------------------------------------------")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

traj = filtered_world.get_traj()
for i, id in enumerate(filtered_ids):
    print(id)
    filtered_world.overlay_trajectory(scenes[0], traj[i], id[0])

In [ ]:
### Scenic Code ###
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.predicate(lambda obj: -67.5 <= (obj.heading - cam.heading) <= 67.5
                                                    and -180 <= (filtered_world.roadDirection(cam.x, cam.y) - cam.heading) <= -90
                                                    and 90 <= (filtered_world.roadDirection(cam.x, cam.y) - cam.heading) <= 180
                                                    and -15 <= (filtered_world.roadDirection(obj.x, obj.y) - obj.heading) <= 15)
                                                             
filtered_world = filtered_world.filter_distance_to_type(distance=10, type="camera")

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
                                                                            and 20 <= (obj.y - cam.y) <= 40,
                                                             type="camera")
                                                             
filtered_world = filtered_world.filter_traj(pred = lambda obj: -5 <= obj.heading <= 5)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg relative to ego

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
                                                                            and 20 <= (obj.y - cam.y) <= 40
                                                                            and -5 <= (obj.heading - cam.heading) <= 5,
                                                             type="camera")
                                                             
filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = Car
# Car offset by (Range(-10, 10), Range(20, 40)), 
# 	facing Range(-5, 5) deg relative to roadDirection

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: -10 <= (obj.x - cam.x) <= 10
                                                                            and 20 <= (obj.y - cam.y) <= 40,
                                                             type="camera")

filtered_world = filtered_world.filter_traj(pred = lambda obj: -5 <= (obj.heading - filtered_world.roadDirection(obj.x, obj.y)) <= 5)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = car on road
# otherCar = Car ahead of ego by Range(4, 19)
# require not (otherCar in intersection)

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam: filtered_world.roadDirection(cam.x, cam.y) == cam.heading 
                                                             and filtered_world.roadType(cam.x, cam.y) != None)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: 4 <= (obj.y - cam.y) <= 19,
                                                             type="camera")

filtered_world = filtered_world.filter_traj(pred = lambda obj: roadType(obj.x, obj.y) != RoadType.Intersection)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# ego = car in intersection
# otherCar = Car ahead of ego by Range(4, 19)
# require otherCar in rightLane

### Apperception Query ###
filtered_world = world.filter_traj_type("vehicle.car")

filtered_world = filtered_world.filter_camera(pred = lambda cam:  filtered_world.roadDirection(cam.x, cam.y) == cam.heading 
                                                              and filtered_world.roadType(cam.x, cam.y) == RoadType.Intersection)

filtered_world = filtered_world.filter_pred_relative_to_type(pred = lambda obj: 4 <= (obj.y - cam.y) <= 19,
                                                             type="camera")

filtered_world = filtered_world.filter_traj(pred = lambda obj: roadType(obj.x, obj.y) == RoadType.RightLane)

filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

In [ ]:
### Scenic Code ###
# def placeObjs(car, numCars):
#     for i in range(numCars):
#         car = Car ahead of car by Range(4, 5)
#         leftCar = Car left of car by Normal(2, 0.1), facing roadDirection
#         rightCar = Car right of car by Normal(3, 0.1), facing Range(0, 10) deg relative to ego.heading
#     return leftCar, rightCar

# spawn_point = 207.26 @ 8.72
# ego = Car at spawn_point, with visible_distance 200

# leftCar, rightCar = placeObjs(ego, 2)
# require (distance to leftCar) < 200
# require (distance to rightCar) < 200


In [ ]:
### Scenic Code ###
# def placeObjs(numPeds):
#     for i in range(numPeds):
#         Pedestrian offset by Range(-5, 5) @ Range(0, 200),
#             facing Range(-120, 120) deg relative to ego.heading

# spawn_point = 207.26 @ 8.72
# ego = Car at spawn_point,
#         with visibleDistance 200

# placeObjs(3)

In [ ]:
### Scenic Code ###
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

In [ ]:
### Scenic Code ###
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

In [ ]:
### Scenic Code ###
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

In [ ]:
### Scenic Code ###
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# point1 = OrientedPoint ahead of ego by Range(0, 40)
# Car at (point1 offset by Range(-1, 1) & 0),
#     facing Range(-15, 15) deg relative to roadDirection

# oppositeCar = Car offset by (Range(-10, -1), Range(0, 50)),
#     facing Range(140, 180) deg relative to ego.heading

# point2 = OrientedPoint ahead of oppositeCar by Range(0, 40)
# Car at (point2 offset by Range(-1, 1) @ 0),
#     facing Range(-15, 15) deg relative to roadDirection

In [ ]:
### Scenic Code ###
# lanesWithRightLane = filter(lambda i: i._laneToRight, network.laneSections)
# egoLane = Uniform(*lanesWithRightLane)

# ego = Car on egoLane,
#         facing Range(-15, 15) deg relative to roadDirection
# cutInCar = Car offset by Range(0, 4) @ Range(0, 5),
#             facing -1*Range(15, 30) deg relative to roadDirection